In [1]:
import json

import numpy as np
import tensorflow as tf
import pandas as pd
from transformers import BertTokenizer, TFBertModel

from rich.progress import track

# distilbert-base-multilingual-cased
# bert-base-multilingual-cased
tokenizer = BertTokenizer.from_pretrained("distilbert-base-multilingual-cased")
bert_model = TFBertModel.from_pretrained("distilbert-base-multilingual-cased")

2023-07-27 11:27:24.421299: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2023-07-27 11:27:24.421319: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2023-07-27 11:27:24.421323: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
2023-07-27 11:27:24.421359: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-07-27 11:27:24.421375: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you 

In [2]:
def get_bert_hidden_state(tokenizer, bert_model, sequence, max_size=None):
    inputs = tokenizer(sequence, return_tensors="tf")
    outputs = bert_model(inputs)
    last_hidden_states = outputs.last_hidden_state
    bert_feat = tf.keras.layers.Flatten()(last_hidden_states).numpy()   
    
    if max_size:
        bert_fixed_feat = np.zeros((1,max_size))
        size = bert_feat.shape[1]
        bert_fixed_feat[0][:size] = bert_feat
    
        return bert_fixed_feat
    else:
        return bert_feat    

### Read marked text inputs

In [3]:
bact_list = pd.read_excel("GPT_bact_activity_list.xlsx", sheet_name="0420 활동명 축약본")
bact_list

,가치 영역 구분,활동
0,대인관계,일기 쓰기
1,대인관계,펍에서 게임 참여하기
2,대인관계,떠다니는 구름 보기
3,대인관계,토론하기
4,대인관계,매니큐어 칠하기
...,...,...
344,자기돌봄,클럽이나 바에 놀러가기
345,자기돌봄,친구를 식사에 초대하기
346,자기돌봄,스스로에게 꽃 선물하기
347,자기돌봄,족욕하기


In [4]:
bact_list['활동'].values.tolist()

['일기 쓰기',
 '펍에서 게임 참여하기',
 '떠다니는 구름 보기',
 '토론하기',
 '매니큐어 칠하기',
 '쉬는 날 계획하기',
 '건설적으로 의견 말하기',
 '친구(들)와 시간 보내기',
 '저녁 외식하기',
 '스파 이용하기',
 '정치적 의견 나누기',
 '몸에 로션 바르기',
 'TV 보기',
 '기프트카드 보내기',
 '베이킹해서 사람들 나눠주기',
 '특별한 음식 요리해 먹기',
 '친구와 영상통화하기',
 '가족과 영상통화하기',
 '욕조에 몸 담그기',
 '악기 연주하기',
 '혼자 시간 보내기',
 '운동하기',
 '누군가에게 호감 표현하기',
 '모임 주최하기',
 '오토바이 타기',
 '와인 시음하기',
 '자연 풍경 감상하기',
 '즉흥적으로 원하는 활동하기',
 '피크닉 가기',
 '머리 감기',
 '미래에 대해 상상하기',
 '화분/식물 관리하기',
 '모임 참석하기',
 '공과금 지불하기',
 '사진 보기',
 '사진 업로드하기',
 '도시 관광하기',
 '박람회 가기',
 '산책하기',
 '관심 주제 조사하기',
 '바다 가기',
 '봉사활동 하기',
 '꽃향기 맡기',
 '잠자기',
 '동물원 가기',
 '수족관 가기',
 '퍼즐 맞추기',
 '자선단체에 물건 기부하기',
 '햇볕 쬐기',
 '마술 배우기',
 '전화 통화하기',
 '팟캐스트 청취하기',
 '라디오 청취하기',
 '수공예 하기',
 '그림 그리기',
 '야외에서 새소리 듣기',
 '무료 강연 듣기',
 '축제 가기',
 '카드놀이 하기',
 '동물보호소에서 봉사하기',
 '좋아하는 영화 다시 보기',
 '샤워하며 노래 부르기',
 '캠핑 가기',
 '배구하기',
 '자전거 타기',
 '대회 참가하기',
 '가로세로 낱말 퀴즈 풀기',
 '반려동물 쓰다듬기',
 '반려동물 껴안기',
 '새로운 요리법 시도하기',
 '집에서 관리하기(홈케어)',
 '과일 음미해보기',
 '차 끓이기',
 '골프치기',
 '누군가에게 호의 베풀기',
 '원하는 물건 직접 만

In [5]:
bact_list.shape

(349, 2)

In [6]:
bact_list['활동'].unique

<bound method Series.unique of 0             일기 쓰기
1       펍에서 게임 참여하기
2        떠다니는 구름 보기
3              토론하기
4          매니큐어 칠하기
           ...     
344    클럽이나 바에 놀러가기
345    친구를 식사에 초대하기
346    스스로에게 꽃 선물하기
347            족욕하기
348    나만의 힐링타임 가지기
Name: 활동, Length: 349, dtype: object>

### Set max fixed tokenized data size

Then get the fixed data

In [7]:
max_token_length = 384

In [8]:
bert_feats = np.zeros((349, max_token_length * 768))

In [9]:
bert_feats.shape

(349, 294912)

In [10]:
inputs = tokenizer(bact_list['활동'].values.tolist(), max_length=max_token_length, padding='max_length', truncation=True, return_tensors='tf')

In [11]:
for key in inputs:
    print (key)

input_ids
token_type_ids
attention_mask


### Batch processing 

In [12]:
batch_size = 32
file_count = len(bact_list['활동'].values.tolist())
for start_index in track(list(range(0, file_count, batch_size))):
    
    end_index = min(start_index + batch_size, file_count)
    input_ids_ = inputs['input_ids'][start_index:end_index]
    token_type_ids_ = inputs['token_type_ids'][start_index:end_index]
    attention_mask_ = inputs['attention_mask'][start_index:end_index]
    
    outputs = bert_model({'input_ids' : input_ids_, 
            'token_type_ids' : token_type_ids_, 
            'attention_mask' : attention_mask_})
    
    bert_feats[start_index:end_index, :] = tf.keras.layers.Flatten()(outputs.last_hidden_state).numpy()  

Output()

In [13]:
bert_feats.shape

(349, 294912)

In [14]:
np.save("bert_feat.npy", bert_feats)

In [15]:
max_size = bert_feats.shape[1]

In [16]:
max_size

294912

### Set up a text search engine

In [41]:
x_bert = tf.keras.Input(shape=(max_size,))
#x = tf.keras.layers.BatchNormalization()(x_bert)
x = tf.keras.layers.Dense(units=bert_feats.shape[0], activation='linear', name='fc1', use_bias=False)(x_bert)   
x_bert_regressor = tf.keras.models.Model(inputs=x_bert, outputs=x)

In [42]:
bert_feats.shape

(349, 294912)

In [43]:
bert_feats_T = bert_feats.T

In [44]:
bert_feats_T.shape

(294912, 349)

In [45]:
temp_weights = x_bert_regressor.get_weights()

In [46]:
temp_weights[-1].shape

(294912, 349)

In [47]:
temp_weights[-1] = bert_feats_T 
x_bert_regressor.set_weights(temp_weights)

In [48]:
x_bert_regressor.save_weights('100-ko-featmatching.h5')

In [49]:
json_model = x_bert_regressor.to_json()
#save the model architecture to JSON file
with open('x_bert_regressor.json', 'w') as json_file:
    json_file.write(json_model)

### 여기서 부터는 RESTful API로 DOCKER에 입력하는 부분

In [78]:
human_input_keyword = '노래를 불렀다.'

In [85]:
# 이 코드는 Docker 안에 있어야 함
input_feats = get_bert_hidden_state(tokenizer, bert_model, human_input_keyword, max_size=max_size)

In [80]:
input_feats

array([[-0.38672116, -0.0322022 , -0.04829908, ...,  0.        ,
         0.        ,  0.        ]])

In [81]:
prediction = x_bert_regressor.predict(input_feats).reshape(-1)

1/1 [==============================] - 0s 14ms/step


In [82]:
total_text = bact_list['활동'].values.tolist()

In [83]:
total_text

['일기 쓰기',
 '펍에서 게임 참여하기',
 '떠다니는 구름 보기',
 '토론하기',
 '매니큐어 칠하기',
 '쉬는 날 계획하기',
 '건설적으로 의견 말하기',
 '친구(들)와 시간 보내기',
 '저녁 외식하기',
 '스파 이용하기',
 '정치적 의견 나누기',
 '몸에 로션 바르기',
 'TV 보기',
 '기프트카드 보내기',
 '베이킹해서 사람들 나눠주기',
 '특별한 음식 요리해 먹기',
 '친구와 영상통화하기',
 '가족과 영상통화하기',
 '욕조에 몸 담그기',
 '악기 연주하기',
 '혼자 시간 보내기',
 '운동하기',
 '누군가에게 호감 표현하기',
 '모임 주최하기',
 '오토바이 타기',
 '와인 시음하기',
 '자연 풍경 감상하기',
 '즉흥적으로 원하는 활동하기',
 '피크닉 가기',
 '머리 감기',
 '미래에 대해 상상하기',
 '화분/식물 관리하기',
 '모임 참석하기',
 '공과금 지불하기',
 '사진 보기',
 '사진 업로드하기',
 '도시 관광하기',
 '박람회 가기',
 '산책하기',
 '관심 주제 조사하기',
 '바다 가기',
 '봉사활동 하기',
 '꽃향기 맡기',
 '잠자기',
 '동물원 가기',
 '수족관 가기',
 '퍼즐 맞추기',
 '자선단체에 물건 기부하기',
 '햇볕 쬐기',
 '마술 배우기',
 '전화 통화하기',
 '팟캐스트 청취하기',
 '라디오 청취하기',
 '수공예 하기',
 '그림 그리기',
 '야외에서 새소리 듣기',
 '무료 강연 듣기',
 '축제 가기',
 '카드놀이 하기',
 '동물보호소에서 봉사하기',
 '좋아하는 영화 다시 보기',
 '샤워하며 노래 부르기',
 '캠핑 가기',
 '배구하기',
 '자전거 타기',
 '대회 참가하기',
 '가로세로 낱말 퀴즈 풀기',
 '반려동물 쓰다듬기',
 '반려동물 껴안기',
 '새로운 요리법 시도하기',
 '집에서 관리하기(홈케어)',
 '과일 음미해보기',
 '차 끓이기',
 '골프치기',
 '누군가에게 호의 베풀기',
 '원하는 물건 직접 만

### Text search engine response

In [84]:
# 여기서 index가 출력임.

k=5

for index in prediction.argsort()[-k:][::-1]:
    print (index, total_text[index])    

120 노래방 가기
133 파티 계획하기
318 맛집 탐방하기
324 휴가 계획하기
221 바비큐 하기
